In [0]:
import pandas as pd
pd.set_option("display.max_colwidth", -1)

In [0]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
from google.colab import drive
drive.mount("/content/drive/")

project_path = "/content/drive/My Drive/StaticalNLP/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [6]:
from gensim.models.keyedvectors import KeyedVectors
w2vec = KeyedVectors.load_word2vec_format(project_path+ "gensim_glove.6B.50d.txt", binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
a = w2vec.word_vec("cat")
b = w2vec.word_vec("dog")

In [0]:
from scipy.stats.stats import pearsonr

In [9]:
pearsonr(a,b)

(0.9213777014333859, 2.4711030233855363e-21)

In [0]:
title1 = "laws of physics"
title2 = "theory of relativity"
title3 = "battle of panipat"

In [0]:
title1_vec = np.array([w2vec.word_vec(x) for x in title1.split(" ")]).mean(axis=0)

In [0]:
title2_vec = np.array([w2vec.word_vec(x) for x in title2.split(" ")]).mean(axis=0)
title3_vec = np.array([w2vec.word_vec(x) for x in title3.split(" ")]).mean(axis=0)

In [14]:
pearsonr(title1_vec, title2_vec)

(0.8310371748765593, 8.078782492695916e-14)

In [0]:
books_data = pd.read_csv(project_path + "day2_snlp/books.csv", encoding='latin')

In [16]:
books_data.head(5)

,title,category
0,"Oral and Maxillofacial Surgery: An Objective-Based Textbook, 2e",Medical Books
1,"Barron's GRE, 21st Edition",Test Preparation
2,George Balanchine: The Ballet Maker (Eminent Lives),Biographies & Memoirs
3,"A Partner in Holiness: Deepening Mindfulness, Practicing Compassion and Enriching Our Lives Through the Wisdom of R. Levi Yitzhak of Berdichev's, Vol. 2 (Institute for Jewish Spirituality)",Religion & Spirituality
4,Construction Scheduling: Principles and Practices (2nd Edition),Arts & Photography


In [17]:
books_data.columns

Index(['title', 'category'], dtype='object')

In [0]:
import re

In [0]:
books_data['title_clean'] = [re.sub("[^a-zA-Z ]", "", x).lower() for x in books_data['title']]

In [0]:
from tqdm import tqdm

In [0]:
title_vec = np.zeros((books_data.shape[0], 50))

In [22]:
for i in tqdm(range(0, books_data.shape[0])):
    words = books_data['title_clean'].iloc[i].split(" ")
    words = [x.strip() for x in words]
    ind_word_vecs = [w2vec.word_vec(x) for x in words if x in w2vec.vocab]
    title_vec[i] = np.array(ind_word_vecs).mean(axis=0)

  0%|          | 0/5700 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice.
  """
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5700/5700 [00:00<00:00, 19242.09it/s]


In [23]:
title_vec.shape

(5700, 50)

In [0]:
title_vec = np.nan_to_num(title_vec)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
cosine_sim_titles = cosine_similarity(title_vec)

In [27]:
cosine_sim_titles.shape

(5700, 5700)

In [29]:
title_id = 13
#title_id = 34 # probelm in results bcz high frequency word edition meesing the results use tfidf to resove it.
books_data['title'].iloc[title_id]

'Organ Transplantation (Health and Medical Issues Today)'

In [30]:
top_n_idx = np.flip(np.argsort(cosine_sim_titles[title_id,:]), axis=0)[0:10]
top_n_sim_values = cosine_sim_titles[title_id, top_n_idx]
top_n_sim_values

array([1.        , 0.92993812, 0.92339052, 0.92211345, 0.91843464,
       0.91842045, 0.91840914, 0.91698754, 0.91646997, 0.9157601 ])

In [0]:
top_n_index = top_n_idx[top_n_sim_values > 0]

In [32]:
books_data['title'].iloc[top_n_index]

13      Organ Transplantation (Health and Medical Issues Today)                                                
616     Medicine and Social Justice: Essays on the Distribution of Health Care                                 
2354    Health Care Emergency Management: Principles and Practice                                              
3125    IBS: A Doctor's Plan for Chronic Digestive Troubles 3 Ed: The Definitive Guide to Prevention and Relief
932     A Core Curriculum for Diabetes Education: Diabetes Education And Program Management                    
3840    The Anticipatory Corpse: Medicine, Power, and the Care of the Dying (ND Studies in Medical Ethics)     
2014    Medicine and Charity Before the Welfare State (Studies in the Social History of Medicine)              
1925    Small Animal Emergency and Critical Care for Veterinary Technicians, 3e                                
1265    Occupational Therapy in Home Health Care                                                        

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vect = TfidfVectorizer(ngram_range=(1,2), stop_words='english', max_features=5000)

In [0]:
vect.fit(books_data['title_clean'])
title_matrix = vect.transform(books_data['title_clean'])

In [41]:
title_matrix.shape

(5700, 5000)

In [0]:
title_matrix = title_matrix.toarray()

In [46]:
np.where(title_matrix[34,:] > 0)[0]

array([1290, 4110])

In [0]:
features = np.where(title_matrix[34,:] > 0)[0]

In [49]:
[vect.get_feature_names()[x] for x in features]

['edition', 'shadowrun']

In [50]:
title_matrix[34, 1290]

0.4337506779996424

In [51]:
title_matrix[34, 4110]

0.901032934656026

In [52]:
idx = 34
feture_names = [vect.get_feature_names()[x] for x in features]
feture_names

['edition', 'shadowrun']

In [53]:
features

array([1290, 4110])

In [0]:
feature_weights = np.array([title_matrix[idx, x] for x in features])[:,np.newaxis]

In [57]:
feature_weights.shape

(2, 1)

In [0]:
word_vec = np.array([w2vec.word_vec(x) if x in w2vec.vocab else np.zeros(50) for x in features])

In [59]:
word_vec.shape

(2, 50)

In [0]:
res = word_vec*feature_weights

In [62]:
res.shape[0]

2

putting all those in function

In [0]:
def get_weighted_vector(idx):
    features = np.where(title_matrix[idx, :] > 0)[0]
    feature_names = [vect.get_feature_names()[x] for x in features]
    feature_weights = np.array([title_matrix[idx,x] for x in features])[:,np.newaxis]
    word_vecs = np.array([w2vec.word_vec(x) if x in w2vec.vocab else np.zeros(50) for x in feature_names])
    res = word_vecs*feature_weights
    return res.mean(axis=0)

In [0]:
from tqdm import tqdm

In [70]:
title_vec_weighted = np.zeros((books_data.shape[0], 50))
for i in tqdm(range(0,books_data.shape[0])):
    vec = get_weighted_vector(i)

    if vec.shape[0] == 0:
        title_vec_weighted[i] = np.zeros(50)
    else:
        title_vec_weighted[i] = vec       

  1%|          | 36/5700 [00:00<01:37, 57.98it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice.
  import sys
100%|██████████| 5700/5700 [01:34<00:00, 60.14it/s]


In [71]:
title_id = 34
books_data['title'].iloc[title_id]

'Shadowrun: Third Edition (FPR25000)'

In [76]:
cosine_sim_titles_weighted = cosine_similarity(title_vec_weighted)
top_n_idx = np.flip(np.argsort(cosine_sim_titles_weighted[title_id,]),axis=0)[0:10]
top_n_sim_values = cosine_sim_titles_weighted[title_id, top_n_idx]
top_n_sim_values

array([1.        , 0.91771819, 0.90715996, 0.86659303, 0.74783249,
       0.74140998, 0.73300601, 0.73190702, 0.73019075, 0.68625087])

In [78]:
top_n_idx = top_n_idx[top_n_sim_values > 0]
books_data['title'].iloc[top_n_idx]

34      Shadowrun: Third Edition (FPR25000)                                                                                                                                                        
4879    Shadowrun 4th Edition                                                                                                                                                                      
2795    Shadowrun Unwired (Shadowrun (Catalyst Hardcover))                                                                                                                                         
4660    Rigger 2: A Shadowrun Sourcebook                                                                                                                                                           
3956    Minecraft Pocket Edition: The Minecraft Pocket Edition Essentials Handbook Guide to Minecraft (An Unofficial Minecraft Pocket Edition Handbook) ... edition, minecraft handbook, minecraft)
274     Sixth World 